# Archivo XML
Un XML está compuesto de tags y atributos. Es posible leer desde un archivo local o a través de una URL mediante librerías como `urllib`.

In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('ejemplo.xml')
print(type(tree))
root = tree.getroot()
root

<class 'xml.etree.ElementTree.ElementTree'>


<Element 'data' at 0x0000025388E15728>

Dentro de root, hay n elementos, considerados como los *children* de root. Cada elemento del XML tiene un *tag* y varios atributos en formato clave-valor.

In [2]:
for child in root:
    print(child.tag)
    print(child.attrib)
    print(child.attrib.get('name') + '\n')

country
{'name': 'Liechtenstein'}
Liechtenstein

country
{'name': 'Singapore'}
Singapore

country
{'name': 'Panama'}
Panama



Para obtener los children de cada línea:

In [3]:
for node in root.iter():
    print(node.tag)

data
country
rank
year
gdppc
neighbor
neighbor
country
rank
year
gdppc
neighbor
country
rank
year
gdppc
neighbor
neighbor


Los elementos van ordenados, por lo que podremos acceder a los mismos mediante su orden.

Si queremos acceder a lo que hay dentro de los tags: `.text`. Devuelve un String, aunque estemos hablando de un número.

In [4]:
print(root[0][1])
print(root[0][1].text)

<Element 'year' at 0x0000025388E4CAE8>
2008


Para iterar sobre unos elementos concretos. Podría haber atributos que se llamen igual en diferentes puntos del XML, y quizá ese no es el comportamiento que deseamos. Podemos filtrar los que nos interesan usando expresiones XPath.

In [5]:
for neighbor in root.iter('neighbor'):
    print(type(neighbor))
    print(neighbor.attrib)
    print(neighbor.attrib['name'])

<class 'xml.etree.ElementTree.Element'>
{'name': 'Austria', 'direction': 'E'}
Austria
<class 'xml.etree.ElementTree.Element'>
{'name': 'Switzerland', 'direction': 'W'}
Switzerland
<class 'xml.etree.ElementTree.Element'>
{'name': 'Malaysia', 'direction': 'N'}
Malaysia
<class 'xml.etree.ElementTree.Element'>
{'name': 'Costa Rica', 'direction': 'W'}
Costa Rica
<class 'xml.etree.ElementTree.Element'>
{'name': 'Colombia', 'direction': 'E'}
Colombia


1. `find` para buscar un tag dentro del árbol inmediato, y no de sus hijos.
2. `iter` para buscar un tag dentro de todo un árbol, incluidos sus hijos.
3. `get` para conseguir el valor de un atributo.

In [6]:
for country in root.findall('country'):
    rank = country.find('rank').text
    neighbor = country.find('neighbor').text
    name = country.get('name')
    
    print(name, neighbor, rank)

Liechtenstein None 1
Singapore None 4
Panama None 68


También podríamos modificarlo

In [7]:
for rank in root.iter('rank'):
    new_rank = int(rank.text) + 1
    rank.text = str(new_rank)
    rank.set('updated', 'yes')

In [8]:
for country in root.findall('country'):
    rank = country.find('rank').text 
    name = country.get('name') 
    print(name, rank)

Liechtenstein 2
Singapore 5
Panama 69


In [9]:
tree.write('output.xml')

O eliminar elementos

In [10]:
for country in root.findall('country'):
    rank = int(country.find('rank').text)
    if rank > 50:
        root.remove(country)

Para escribir los datos en un nuevo XML

In [11]:
tree.write('output.xml')

Cada elemento que encuentra `findall()` o `iter()` es un `Element`, sobre el que podemos acceder a sus datos gracias a atributos como como `tag` o `attrib`.

Acceso mediante XPath. Ya que la estructura de los XML es un árbol de tags, podremos acceder a los elementos mediante la ruta relativa de los tags.

In [12]:
print(root.findall('./country/neighbor'))

[<Element 'neighbor' at 0x0000025388E53A48>, <Element 'neighbor' at 0x0000025388E53A98>, <Element 'neighbor' at 0x0000025388E53C28>]


# XML con RSS
RSS es una manera que tienen las páginas web de publicar su contenido. En este caso no es un HTML, ni un CSS como se hace habitualmente, sino que será un XML, con un árbol de tags y distinta información. Páginas que utilizan esto son periódicos, foros o blogs. Permite acceder a los titulares de noticias, tanto de las generales, como de las secciones del periódico, de tal manera que puedas monitorizarlos en una aplicación. Los datos son abiertos y el formato de publicación es XML.

En nuestro caso vamos a desarrollar un programa mediante el que recogeremos el RSS del periódico *El Pais* y montaremos una tabla con las principales noticias.

In [13]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

var_url = urlopen('http://ep00.epimg.net/rss/tags/ultimas_noticias.xml')
xmldoc = parse(var_url)

for item in xmldoc.findall('./channel/item'):
    title = item.findtext('title')
    date = item.findtext('pubDate')
    link = item.findtext('link')
    
    print(title, date, link)

Al menos 13 personas perdieron la vida en una masacre en Guatemala Mon, 20 Dec 2021 20:46:26 +0100 https://elpais.com/elpais/2021/12/20/album/1640025174_409640.html#?ref=rss&format=simple&link=link
El legado invisible de Castells Mon, 20 Dec 2021 20:21:05 +0100 https://elpais.com/sociedad/2021/12/20/actualidad/1640001521_117817.html#?ref=rss&format=simple&link=link
Desactivar el suelo del sur de Irak Mon, 20 Dec 2021 18:42:57 +0100 https://elpais.com/elpais/2021/12/15/album/1639578225_563487.html#?ref=rss&format=simple&link=link
‘Desiertos’ Mon, 20 Dec 2021 17:58:32 +0100 https://elpais.com/elpais/2021/12/20/album/1640019004_037258.html#?ref=rss&format=simple&link=link
La migración como arma arrojadiza Mon, 20 Dec 2021 09:48:23 +0100 https://elpais.com/elpais/2021/12/17/album/1639752613_831789.html#?ref=rss&format=simple&link=link
Los destinos más esperados en 2022: del Caribe a Finlandia Mon, 20 Dec 2021 08:31:39 +0100 https://elpais.com/elviajero/2021/12/17/actualidad/1639733347_7557

Enlaces documentación

https://docs.python.org/3/library/xml.etree.elementtree.html

https://rico-schmidt.name/pymotw-3/xml.etree.ElementTree/parse.html